# Use Drosdowsky (1997) classification on sounding data

In [1]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
import numpy as np
import numpy.ma as ma
from netCDF4 import Dataset
from datetime import datetime, timedelta
import glob
from copy import deepcopy
import math
from matplotlib import pyplot as plt
from matplotlib import dates
%matplotlib inline
import metpy.calc as mpcalc

In [2]:
data_path_sounding = '/home/rjackson/data/soundings/'

In [9]:
# get_sounding)times
#     start_year = Start year of animation
#     start_month = Start month of animation
#     start_day = Start day of animation
#     start_hour = Start hour of animation
#     end_year = End year of animation
#     end_month = End month of animation
#     end_day = End day of animation
#     end_minute = End minute of animation
#     minute_interval = Interval in minutes between scans (default is 5)
# This procedure acquires an array of sounding times between start_time and end_time. 
# Only 23 UTC is loaded following Pope et al. (2008)
def get_sounding_times(start_year, start_month, start_day,
                       start_hour, start_minute, end_year,
                       end_month, end_day, end_hour, 
                       end_minute, minute_interval=5):

    start_time = datetime(start_year,
                          start_month,
                          start_day,
                          start_hour,
                          start_minute,
                          )
    end_time = datetime(end_year,
                        end_month,
                        end_day,
                        end_hour,
                        end_minute,
                        )
    deltatime = end_time - start_time

    if(deltatime.seconds > 0 or deltatime.minutes > 0):
        no_days = deltatime.days + 1
    else:
        no_days = deltatime.days
    
    if(start_day != end_day):
        no_days = no_days + 1
        
    days = np.arange(0, no_days, 1)
    print('We are about to load sounding files for ' + str(no_days) + ' days')
    
    # Find the list of files for each day
    cur_time = start_time

    file_list = []
    time_list = []
    for i in days:
        year_str = "%04d" % cur_time.year
        day_str = "%02d" % cur_time.day
        month_str = "%02d" % cur_time.month
        format_str = (data_path_sounding +
                      'twpsondewnpnC3.b1.' +
                      year_str +
                      month_str +
                      day_str +
                      '*.23*'
                     '*custom.cdf')
    
          
        data_list = glob.glob(format_str)
        if(i % 100 == 0):
            print(i)
        for j in range(0, len(data_list)):
            file_list.append(data_list[j])
        cur_time = cur_time + timedelta(days=1)
        
   
    # Parse all of the dates and time in the interval and add them to the time list
    past_time = []
    for file_name in file_list:
        date_str = file_name[-26:-11]  
        year_str = date_str[0:4]
        month_str = date_str[4:6]
        day_str = date_str[6:8]
        hour_str = date_str[9:11]
        minute_str = date_str[11:13]
        second_str = date_str[13:15]
                
        cur_time = datetime(int(year_str),
                            int(month_str),
                            int(day_str),
                            int(hour_str),
                            int(minute_str),
                            int(second_str))
        time_list.append(cur_time)
    
    # Sort time list and make sure time are at least xx min apart
    time_list.sort()
    time_list_sorted = deepcopy(time_list)
   
    time_list_final = []
    past_time = []
    
    for times in time_list_sorted: 
        
        cur_time = times  
        
        if(past_time == []):
            past_time = cur_time
            
        if(cur_time - past_time >= timedelta(minutes=minute_interval)
           and cur_time >= start_time and cur_time <= end_time):
            
            time_list_final.append(cur_time)
            past_time = cur_time
               
    return time_list_final

# Get a Radar object given a time period in the CPOL dataset
def get_sounding(time):
    year_str = "%04d" % time.year
    month_str = "%02d" % time.month
    day_str = "%02d" % time.day
    hour_str = "%02d" % time.hour
    minute_str = "%02d" % time.minute
    second_str = "%02d" % time.second
    file_name_str = (data_path_sounding +
                     'twpsondewnpnC3.b1.' +
                     year_str +
                     month_str +
                     day_str +
                     '.' +
                     hour_str +
                     minute_str +
                     second_str +
                     '.custom.cdf')
    sounding = Dataset(file_name_str, mode='r')
    return sounding

Load netCDF sounding data

surface (assumed
here to be 1013 hPa), 950, 925, 900, 850, 800, 750, 700,
650, 600, 550, 500, 400, 300, 200, and 100 hPa (a total of
16 levels)

In [4]:
def find_nearest(array,value):
    idx = (np.abs(array-value)).argmin()
    return idx

Here is where we input the times and pressure levels to get sounding from.

In [10]:
start_year = 2002
end_year = 2015

sounding_times = get_sounding_times(start_year,1,1,0,1,
                                    end_year,1,1,23,1)

pres_levels = [1013, 950, 925, 900, 850, 800, 750, 700, 
               650, 600, 550, 500, 400, 300, 200, 100] 
print(len(sounding_times))

We are about to load sounding files for 4749 days
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4493


Loop over all of the soundings.

In [11]:
## Save soundings at 16 levels for later

u_soundings = np.zeros((len(sounding_times),16))
v_soundings = np.zeros((len(sounding_times),16))
t_soundings = np.zeros((len(sounding_times),16))
z_soundings = np.zeros((len(sounding_times),16))
dp_soundings = np.zeros((len(sounding_times),16))
time_soundings = []
pres_soundings = np.zeros((len(sounding_times),16))
count = 0
no_masked = 0

for time in sounding_times:
    if(time.month <= 4 or time.month >= 9):
        if(time.hour == 23):
            pres_index = []
            # Load sounding file
            Sounding_netcdf = get_sounding(time)

            # Convert timestamps to datetime format        
            p = Sounding_netcdf.variables['pres'][:]
            u = Sounding_netcdf.variables['u_wind'][:]
            v = Sounding_netcdf.variables['v_wind'][:]
            t = Sounding_netcdf.variables['tdry'][:]
            z = Sounding_netcdf.variables['alt'][:]
            dp = Sounding_netcdf.variables['dp'][:]
            
            # Take levels from the sounding and place them into the array
            
            for pres in pres_levels:
                pres_index.append(find_nearest(p,pres))
           
            
            # Check for availability of 16 pressure levels
            good_sounding = 1
            for i in range(0,len(pres_levels)-1):
                if(p[pres_index[i]] < pres_levels[i]-20 or
                    p[pres_index[i]] > pres_levels[i]+20):
                    good_sounding = 0
                if(abs(u[pres_index[i]]) > 75 or
                   abs(v[pres_index[i]]) > 75):
                    good_sounding = 0
            
            u = u[pres_index]
            v = v[pres_index]
            t = t[pres_index]
            z = z[pres_index]
            dp = dp[pres_index]
            
            for i in range(0,len(u)):
                if(u[i] < -75 or v[i] < -75 or
                   u[i] is np.ma.masked or
                   v[i] is np.ma.masked or
                   dp[i] is np.ma.masked or
                   t[i] is np.ma.masked or
                   dp[i] < -99):
                    good_sounding = 0
            
            if(t[0] < 0):
                t[:] = float('nan')
                good_sounding = 0
                
            # If pressure levels are not available, mask the entire sounding
            if(good_sounding == 0):
                no_masked = no_masked + 1
            else:    
                u_soundings[count][:] = u
                v_soundings[count][:] = v
                t_soundings[count][:] = t
                dp_soundings[count][:] = dp
                z_soundings[count][:] = z
                time_soundings.append(time)
                pres_soundings[count][:] = pres_levels
                count = count + 1   
                
if(count % 100 == 0):
    print(time)

u_soundings = u_soundings[0:count-1][:]
v_soundings = v_soundings[0:count-1][:]
t_soundings = t_soundings[0:count-1][:]
dp_soundings = dp_soundings[0:count-1][:]
z_soundings = z_soundings[0:count-1][:]
pres_soundings = pres_soundings[0:count-1][:]
print(str((no_masked/(count+no_masked)*100)) + '% of soundings masked')

12.589316093909492% of soundings masked


## Time-height sections

Test the Drosdowsky (1997) classification over the two months of TWP-ICE

In [ ]:
start_year = 2006
start_month = 1
start_day = 1

start_time = datetime(year=start_year, 
                      month=start_month, 
                      day=start_day)
end_year = 2006
end_month = 3
end_day = 1

end_time = datetime(year=end_year, 
                    month=end_month, 
                    day=end_day)

indicies = []
datenums = []
i = 0
for times in time_soundings:
    if(times >= start_time and times <= end_time):
        indicies.append(i)
        datenums.append(dates.date2num(times))
                        
    i = i + 1
    
datelocs = [datetime.strptime('2006-01-01', "%Y-%m-%d"),
            datetime.strptime('2006-01-15', "%Y-%m-%d"),
            datetime.strptime('2006-02-01', "%Y-%m-%d"),
            datetime.strptime('2006-02-15', "%Y-%m-%d")]

x = dates.date2num(datelocs)
plt.figure(figsize=(8,20))
plt.subplot(311)
X,Y = np.meshgrid(pres_levels, datenums)
Z = u_soundings[indicies][:]
CS = plt.contour(Y, X, Z)

plt.clabel(CS, inline=1, fontsize=10)
plt.title('U (m/s)')
plt.gca().set_xticks(x)

# Set the xtick labels to correspond to just the dates you entered.
plt.gca().set_xticklabels([date.strftime("%Y-%m-%d") for date in datelocs])

# Calculate the pressure weighted DLM for each timestep
DLM = np.zeros(len(indicies))
U_300100 = np.zeros(len(indicies))
j = 0

for i in indicies:
    DLM[j] = sum(pres_levels[0:11]*u_soundings[i,0:11]/(sum(pres_levels[0:11])))
    U_300100[j] = np.mean(u_soundings[i,13:15])
    j = j + 1

classification_Drosdowsky = np.zeros(len(DLM))
j = 0
N = 1
break_spell = 0
U = 2.5
for i in range(0,len(DLM)):
    if(np.mean(DLM[i-N:i]) > U*(N+1)/N and DLM[i] > 0):
        if(U_300100[i] < 0):
            N = N + 1
        else:
            if(N > 1):
                classification_Drosdowsky[j-N+1:j-1] = 1
                N = 1
    else:
        if(N > 1):
            classification_Drosdowsky[j-N+1:j-1] = 1
            N = 1
    j = j + 1
            
plt.subplot(312)
plt.plot_date(datenums,DLM)

plt.xlabel('Date')
plt.ylabel('DLM U Surface-500 hPa [m/s]')

plt.subplot(313)
plt.plot_date(datenums,classification_Drosdowsky)
plt.xlabel('Date')
plt.ylabel('Classification')
plt.gca().set_yticks([0,1])
plt.gca().set_yticklabels(['Break', 'Monsoon'])
plt.ylim((-1,2))

## Write classification from 2003 to 2015 to netCDF file

In [16]:
file_path = '/home/rjackson/data/Drosdowsky.cdf'

indicies = []
datenums = []
i = 0
for times in time_soundings:
    indicies.append(i)
    datenums.append(dates.date2num(times)) 
    i = i + 1
    
# Calculate the pressure weighted DLM for each timestep
DLM = np.zeros(len(indicies)-1)
U_300100 = np.zeros(len(indicies)-1)
year = np.zeros(len(indicies)-1)
month = np.zeros(len(indicies)-1)
day = np.zeros(len(indicies)-1)
j = 0

for i in range(0,len(indicies)-1):
    DLM[j] = sum(pres_levels[0:11]*u_soundings[i,0:11]/(sum(pres_levels[0:11])))
    U_300100[j] = np.mean(u_soundings[i,13:15])
    year[j] = time_soundings[i].year
    month[j] = time_soundings[i].month
    day[j] = time_soundings[i].day
    j = j + 1

classification_Drosdowsky = np.zeros(len(DLM))
j = 0
N = 1
break_spell = 0
U = 2.5
# Calculate Deep Layer Mean wind and classify time periods

for i in range(0,len(DLM)-1):
    if(np.mean(DLM[i-N:i]) > U*(N+1)/N and DLM[i] > 0):
        if(U_300100[i] < 0):
            N = N + 1
        else:
            if(N > 1):
                classification_Drosdowsky[j-N+1:j-1] = 1
                N = 1
    else:
        if(N > 1):
            classification_Drosdowsky[j-N+1:j-1] = 1
            N = 1
    j = j + 1
    
out_netcdf = Dataset(file_path, mode='w')            
out_netcdf.createDimension('time', len(classification_Drosdowsky))

print(len(classification_Drosdowsky))
groups_file = out_netcdf.createVariable('groups', 'i4', ('time',))
groups_file.long_name = 'Pope classification regime'
groups_file.units = '0 = Break, 1 = Monsoon'
groups_file[:] = classification_Drosdowsky

years_file = out_netcdf.createVariable('year', year.dtype, ('time',))
years_file.long_name = 'Year'
years_file.units = 'YYYY'
years_file[:] = year

month_file = out_netcdf.createVariable('month',  month.dtype, ('time',))
month_file.long_name = 'Month'
month_file.units = 'MM'
month_file[:] = month

day_file = out_netcdf.createVariable('day', day.dtype, ('time',))
day_file.long_name = 'Day'
day_file.units = 'DD'
day_file[:] = day
                                       
out_netcdf.close()

2568


/home/rjackson/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/home/rjackson/anaconda3/lib/python3.5/site-packages/numpy/core/_methods.py:70: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
